In [ ]:
# 利用pytorch对minist进行识别
import torch
import numpy
from torchvision import datasets,transforms
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.device_count() 

1

In [ ]:
torch.cuda.current_device()

0

In [ ]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5],std=[0.5])])
data_train=datasets.MNIST(root="./data",  transform=transform, train=True,download=True)
data_test=datasets.MNIST(root="./data", transform=transform, train=False)

In [ ]:
# 进行数据装载
data_loader_train = torch.utils.data.DataLoader(dataset=data_train,
                                                batch_size = 64,
                                                shuffle = True)

data_loader_test = torch.utils.data.DataLoader(dataset=data_test,
                                               batch_size = 64,
                                               shuffle = True)

In [ ]:
# 通过tensorflow获取mnist

In [ ]:
class MnistLayer(torch.nn.Module):
  def __init__(self):
    super(MnistLayer,self).__init__()
    # 28 * 28
    self.conv1 = nn.Conv2d(1,64,3,stride=1,padding=1)
    self.conv2 = nn.Conv2d(64,128,3,stride=1,padding=1)
    self.max_pool = nn.MaxPool2d(2,stride=2)

    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(7*7*128,256)
    self.fc2 = nn.Linear(256,64)
    self.fc3 = nn.Linear(64,10)
    self.drop = nn.Dropout2d(0.5)
    self.flatten = nn.Flatten()

  def forward(self,x):
    # x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    # print(x.shape)
    x = self.conv1(x)
    # print("self.conv1(x)",x.shape)
    x = self.relu(x)
    # print("self.relu(x)",x.shape)
    x = self.max_pool(x)
    # print("self.max_pool(x)",x.shape)
    x = self.conv2(x)
    # print("self.conv2(x)",x.shape)
    x = self.relu(x)
    # print("self.relu(x)",x.shape)
    x = self.max_pool(x)
    # print("self.max_pool(x)",x.shape)
    x = self.flatten(x)
    # print("self.flatten(x)",x.shape)
    x = F.relu(self.fc1(x))
    self.drop(x)
    x = F.relu(self.fc2(x))
    self.drop(x)
    return self.fc3(x)

In [ ]:
mnist = MnistLayer()
mnist = mnist.cuda()
print(mnist)

MnistLayer(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=6272, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (drop): Dropout2d(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [ ]:
optimizer = optim.Adam(mnist.parameters(),lr=0.001)

In [ ]:
# define cost function
cost = nn.CrossEntropyLoss()

In [ ]:
n_epochs = 5

In [ ]:
for epoch in range(n_epochs):
  running_loss = 0.0
  running_correct = 0.0
  print("Epoch {}/{}".format(epoch, n_epochs))
  for data in data_loader_train:
    X_train,y_train = data
    X_train = X_train.cuda()
    y_train = y_train.cuda()
    out = mnist(X_train)
    # print(out.shape)
    # print(out)
    # print()
    _,pred = torch.max(out,1)
    optimizer.zero_grad()#清除梯度
    loss = cost(out,y_train)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    running_correct += torch.sum(pred == y_train.data)
  testing_correct = 0
  for data in data_loader_test:
    X_test, y_test = data
    X_test = X_test.cuda()
    y_test = y_test.cuda()
    outputs = mnist(X_test)
    _, pred = torch.max(outputs.data, 1)
    testing_correct += torch.sum(pred == y_test.data)
  print("Loss is:{:.4f}, Train Accuracy is:{:.4f}%, Test Accuracy is:{:.4f}".format(running_loss/len(data_train),100*running_correct/len(data_train),100*testing_correct/len(data_test)))

Epoch 0/5
Loss is:0.0022, Train Accuracy is:95.6983%, Test Accuracy is:98.3200
Epoch 1/5
Loss is:0.0007, Train Accuracy is:98.6567%, Test Accuracy is:98.8400
Epoch 2/5
Loss is:0.0005, Train Accuracy is:99.0183%, Test Accuracy is:99.0800
Epoch 3/5
Loss is:0.0003, Train Accuracy is:99.2417%, Test Accuracy is:99.1800
Epoch 4/5
Loss is:0.0002, Train Accuracy is:99.5050%, Test Accuracy is:99.1700
